# Cost-Benefit Analysis 

Analyzing the decision to install a solar panel.  
What has been predicted? - The median cost of installing a solar panel in particular U.S. zipcodes.  
What do we want to find out - What is the value of installing a solar panel?  
 - What is the value of the electricity generated by the PV installation?
 - What is the payback time - how many years?

A Note about Units:
- Actual and Predicted Cost of Installation - U.S. Dollars
- Expected_PV_annual - kWh
- Res_Rate and Comm_Rate - $/kWh  
- Size of Solar System - kW
- Res and Comm Consumption (Monthly and Annual) - kWh
- Res and Comm Payback Time - Years
- Energy Value Res and Comm - U.S. Dollars
- Yearly Electric Bill No Solar - U.S. Dollars
- Median Solar Insolation (Sun Exposure) - kWh/m2


In [68]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import pickle
from pprint import pprint
#import fancyimpute
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, learning_curve
from scipy import stats
sns.set_style('darkgrid')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [69]:
pd.set_option('max_columns', 200)

In [70]:
solar_predictions = pd.read_pickle('./prediction_results.pkl')

In [71]:
electric_rates = pd.read_pickle('./electric_rates.pkl')

In [72]:
electric_rates.head()

,zipcode,comm_rate,ind_rate,res_rate
0,01001,0.126099,0.101999,0.145852
1,01002,0.119927,0.107994,0.142541
2,01003,0.126099,0.101999,0.145852
3,01004,0.126099,0.101999,0.145852
4,01005,0.113755,0.113988,0.139230


In [73]:
df = pd.merge(solar_predictions, electric_rates, how='left', on='zipcode')

In [74]:
df[df.zipcode == '10001']

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,comm_rate,ind_rate,res_rate
743,25729.965,17858.476973,7871.488027,10001,40.750633,-73.997177,4158.475,3.275,NY,0.145384,0.119325,0.211082


In [75]:
utility_rate_zips_needed = df[df.res_rate.isnull()].zipcode.tolist()

In [76]:
noniou_rates = pd.read_pickle('./noniou_rates.pkl')

## Grab missing rates from Non investor owned utility dataset.

In [77]:
print(f"Number of Missing Rates: {len(utility_rate_zips_needed)}")

Number of Missing Rates: 469


In [78]:
rates_needed = noniou_rates[noniou_rates.zipcode.isin(utility_rate_zips_needed)]

In [79]:
rates_needed.head(3)

,zipcode,comm_rate,ind_rate,res_rate
185,03226,0.132881,0.103087,0.153882
196,03245,0.132881,0.103087,0.153882
201,03254,0.132881,0.103087,0.153882


In [80]:
df = pd.merge(df, rates_needed, how='left', on='zipcode')

In [82]:
df.replace(np.nan, 0, inplace=True)

In [83]:
df['comm_rate'] = df.comm_rate_x + df.comm_rate_y
df['ind_rate'] = df.ind_rate_x + df.ind_rate_y
df['res_rate'] = df.res_rate_x + df.res_rate_y

In [84]:
df.drop(['comm_rate_x', 'ind_rate_x', 'res_rate_x', 'comm_rate_y', 'ind_rate_y', 'res_rate_y'], 
        axis=1, inplace=True)

In [85]:
df.comm_rate.replace(0, np.nan, inplace=True)
df.ind_rate.replace(0, np.nan, inplace=True)
df.res_rate.replace(0, np.nan, inplace=True)

In [86]:
del df['ind_rate']

In [88]:
print(df[df.state == 'MA'].res_rate.median())
print(df[df.state == 'CA'].res_rate.median())
print(df[df.state == 'MA'].comm_rate.median())
print(df[df.state == 'CA'].comm_rate.median())

0.14548285706151617
0.14084994866818065
0.11674939089150109
0.1305972524675068


In [89]:
df[df.res_rate.isnull()]

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,comm_rate,res_rate
344,43409.53,39498.104645,3911.425355,02557,41.441713,-70.576143,9494.47,7.24,MA,NaN,NaN
2556,5186.83,13694.790284,-8507.960284,91729,34.099175,-117.563934,1955.60,1.26,CA,NaN,NaN


In [90]:
df.at[344, 'comm_rate'] = 0.11674939089150109
df.at[344, 'res_rate'] = 0.14548285706151617
df.at[2556, 'comm_rate'] = 0.1305972524675068
df.at[2556, 'res_rate'] = 0.14084994866818065

In [94]:
df[df.zipcode == '10510']

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,comm_rate,res_rate
768,40724.3,41407.72896,-683.42896,10510,41.139472,-73.835704,11797.16,9.0,NY,0.145384,0.211082


In [95]:
df.state.value_counts()

CA    1299
NY     719
MA     409
AZ     241
CT     228
MD     201
NH     106
TX     101
NV      86
WI      71
NM      68
OR      55
DE      48
MN      19
FL      18
UT       9
IL       7
Name: state, dtype: int64

# State Level Electricity Usage - Residential and Commercial

In [96]:
usage = pd.read_pickle('./electricity_usage.pkl')

In [97]:
df_finances = pd.merge(df, usage, how='left', on='state')

In [98]:
df_finances['avg_annual_consumption_comm'] = df_finances.avg_monthly_consumption_comm * 12
df_finances['avg_annual_consumption_res'] = df_finances.avg_monthly_consumption_res * 12
df_finances['Residential_Payback_Time'] = (df_finances['Predicted'] / df_finances['res_rate'] / df_finances['avg_annual_consumption_res'])
df_finances['Commercial_Payback_Time'] = (df_finances['Predicted'] / df_finances['comm_rate'] / df_finances['avg_annual_consumption_comm'])
df_finances['Annual_Energy_Value_res'] = df_finances['Expected_annual_pv_prod_median'] * df_finances['res_rate']
df_finances['Annual_Energy_Value_comm'] = df_finances['Expected_annual_pv_prod_median'] * df_finances['comm_rate']

In [105]:
df_finances.head(3)

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,comm_rate,res_rate,avg_monthly_consumption_comm,avg_monthly_consumption_res,avg_annual_consumption_comm,avg_annual_consumption_res,Residential_Payback_Time,Commercial_Payback_Time,Annual_Energy_Value_res,Annual_Energy_Value_comm
0,32187.00,34823.748660,-2636.748660,01001,42.062368,-72.625754,8583.645,7.42,MA,0.126099,0.145852,5364,602,64368,7224,33.051033,4.290362,1251.942811,1082.388888
1,30155.24,31275.459039,-1120.219039,01002,42.364061,-72.458739,7558.890,6.24,MA,0.119927,0.142541,5364,602,64368,7224,30.372911,4.051507,1077.451099,906.515324
2,28370.09,34853.454835,-6483.364835,01005,42.418848,-72.106598,8487.920,7.20,MA,0.113755,0.139230,5364,602,64368,7224,34.652628,4.759978,1181.770654,965.544226


# Market Data (OpenPV and Tax Information from IRS)

In [106]:
df_market = pd.read_pickle("./pv_data_3.pkl")

In [107]:
df_market.zipcode.nunique()

11763

In [108]:
df_finances.drop(['avg_monthly_consumption_comm', 'comm_rate', 
                  'avg_annual_consumption_comm', 'Commercial_Payback_Time',
                 'Annual_Energy_Value_comm'], axis=1, inplace=True)

In [109]:
df_finances['yearly_electric_bill_no_solar'] = df_finances['res_rate'] * df_finances['avg_annual_consumption_res']

In [110]:
df_finances[df_finances.zipcode == '90031']

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,res_rate,avg_monthly_consumption_res,avg_annual_consumption_res,Residential_Payback_Time,Annual_Energy_Value_res,yearly_electric_bill_no_solar
2355,16793.86,15891.129319,902.730681,90031,34.085784,-118.206666,4792.185,3.195,CA,0.15384,557,6684,15.454273,737.230518,1028.267644


In [111]:
df_finances[df_finances.zipcode == '10510']

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,res_rate,avg_monthly_consumption_res,avg_annual_consumption_res,Residential_Payback_Time,Annual_Energy_Value_res,yearly_electric_bill_no_solar
768,40724.3,41407.72896,-683.42896,10510,41.139472,-73.835704,11797.16,9.0,NY,0.211082,601,7212,27.200358,2490.167375,1522.322924


In [112]:
df_finances['Savings_25_years'] = df_finances['Annual_Energy_Value_res'] * 25

In [113]:
df_finances.Savings_25_years.median()

28259.1600875409

In [114]:
df_market.head(3)

,state,date_installed,incentive_prog_names,type,size_kw,zipcode,install_type,installer,adjusted_cost,adj_cost_per_watt,lbnl_tts,city,utility_clean,county,annual_PV_prod,annual_insolation,rebate,sales_tax_cost,Agricultural,Commercial,Educational,Government,Nonprofit,Utility,month,year,1,2,3,4,5,6,7,8,9,10,11,annual_pv_imputed,AK,AL,AR,AZ,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2016,2017,2018,latitude,longitude,num_total_returns,num_single_returns,num_married_returns,num_headhouse_returns,num_exemptions,num_dependents,adjust_gross_income,num_return_salaries_wages,salaries_wages_amount,num_returns_taxable_interest,taxable_interest_amount,num_returns_ordinary_dividends,ordinary_dividends_amount,num_returns_qualified_dividends,qualified_dividends_amount,num_returns_business_prof_net_income,business_prof_net_income_amount,num_farm_returns,num_returns_net_capital_gain,net_capital_gain_amount,num_returns_retirement_arrangements,taxable_retirement_arrangements,num_returns_pensions,taxable_pensions_amount,num_returns_unemployment,unemployment_amount,num_returns_SS_benefits,SS_amount,num_returns_self_employment_retirement,self_employment_retirement_amount,num_returns_itemized_deductions,amount_AGI_itemized_returns,total_itemized_deductions_amount,num_returns_state_local_income_taxes,state_local_income_taxes_amount,num_returns_state_local_sales_tax,state_local_sales_tax_amount,num_returns_real_estate_taxes,real_estate_taxes_amount,num_returns_taxes_paid,total_taxes_paid,num_returns_mortgage_interest_paid,mortgage_interest_paid_amount,num_returns_contributions,contributions_amount,num_returns_taxable_income,taxable_income_amount,num_returns_total_tax_credits,total_tax_credits_amount,num_returns_child_tax_credit,child_tax_credit_amount,num_returns_child_care,child_dependent_care_amount,num_returns_residential_energy_tax_credits,residential_energy_tax_credit,num_returns_earned_income_credit,earned_income_credit_amount,num_returns_excess_earned_income_credit,excess_earned_income_credit_refundable,num_returns_alt_min_tax,alt_min_tax_amount,num_returns_income_tax,income_tax_amount,num_returns_tax_liability,total_tax_liability_amount,num_returns_tax_due_filing,tax_due_time_filing_amount,num_returns_overpayments_refunded,overpayments_refunded_amount
0,TX,2013-12-12,Austin Energy,0,10.56,78702,Residential,NaN,32920.68,3.12,1,Austin,Austin Energy,Travis,14999.83,5.16,15000.0,1151.69,0,0,0,0,0,0,12,2013,0,0,0,0,0,0,0,0,0,0,0,14999.83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,30.263378,-97.714483,10860.0,6760.0,1890.0,2030.0,18790.0,6310.0,418160.0,9520.0,336303.0,1750.0,840.0,1090.0,3761.0,990.0,2780.0,2150.0,27599.0,0.0,890.0,17709.0,290.0,3361.0,940.0,15033.0,510.0,2837.0,410.0,3259.0,30.0,417.0,1730.0,157718.0,29602.0,110.0,645.0,1270.0,1791.0,1490.0,6914.0,1690.0,9924.0,1400.0,12240.0,1100.0,2820.0,7900.0,268913.0,2650.0,2217.0,1290.0,1220.0,270.0,125.0,90.0,29.0,3040.0,6638.0,2600.0,5820.0,50.0,242.0,6850.0,44213.0,7760.0,48368.0,1840.0,6681.0,8770.0,20986.0
1,TX,2014-07-11,Austin Energy,0,12.72,78702,Residential,NaN,42362.13,3.33,1,Austin,Austin Energy,Travis,17811.57,5.09,15000.0,1507.17,0,0,0,0,0,0,7,2014,0,0,0,0,0,0,1,0,0,0,0,17811.57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,30.263378,-97.714483,10860.0,6760.0,1890.0,2030.0,18790.0,6310.0,418160.0,9520.0,336303.0,1750.0,840.0,1090.0,3761.0,990.0,2780.0,2150.0,27599.0,0.0,890.0,17709.0,290.0,3361.0,940.0,15033.0,510.0,2837.0,410.0,3259.0,30.0,417.0,1730.0,157718.0,29602.0,110.0,645.0,1270.0,1791.0,1490.0,6914.0,1690.0,9924.0,1400.0,12240.0,1100.0,2820.0,7900.0,268913.0,2650.0,2217.0,1290.0,1220.0,270.0,125.0,90.0,29.

In [115]:
zips_to_use = df_finances.zipcode.tolist()
zips_we_use = df_market[df_market.zipcode.isin(zips_to_use)]

In [116]:
df_finances['median_solar_insolation'] = pd.Series(zips_we_use.groupby('zipcode')['annual_insolation'].median().values)

In [118]:
counties = zips_we_use.groupby(['zipcode', 'county'])['annual_insolation'].median().reset_index().drop_duplicates(subset='zipcode')


In [119]:
df_finances['County'] = pd.Series(counties.county.values)

In [120]:
df_finances.head(3)

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,res_rate,avg_monthly_consumption_res,avg_annual_consumption_res,Residential_Payback_Time,Annual_Energy_Value_res,yearly_electric_bill_no_solar,Savings_25_years,median_solar_insolation,County
0,32187.00,34823.748660,-2636.748660,01001,42.062368,-72.625754,8583.645,7.42,MA,0.145852,602,7224,33.051033,1251.942811,1053.635707,31298.570284,4.24,Hampden
1,30155.24,31275.459039,-1120.219039,01002,42.364061,-72.458739,7558.890,6.24,MA,0.142541,602,7224,30.372911,1077.451099,1029.715572,26936.277474,4.26,Hampshire
2,28370.09,34853.454835,-6483.364835,01005,42.418848,-72.106598,8487.920,7.20,MA,0.139230,602,7224,34.652628,1181.770654,1005.795437,29544.266342,4.39,Worcester


In [121]:
pd.to_pickle(df_finances, './finances.pkl')

# Add in all Solar Information and Incentive Information 

From Project Sunroof Scraped Data

In [122]:
df_solar = pd.read_pickle("./solar_numbers.pkl")

In [123]:
df_finances = pd.merge(df_finances, df_solar, how='left', on='zipcode')

In [128]:
df_finances[df_finances.zipcode == '90031']

,Actual,Predicted,Residual,zipcode,latitude,longitude,Expected_annual_pv_prod_median,Size_System_median_kw,state,res_rate,avg_monthly_consumption_res,avg_annual_consumption_res,Residential_Payback_Time,Annual_Energy_Value_res,yearly_electric_bill_no_solar,Savings_25_years,median_solar_insolation,County,Solar_Incentives,existing_solars,viable_solar_roofs,total_viable_roofs,total_sq_foot_roof,total_capacity,total_production,roof_space_per_roof,capacity_per_roof,production_per_roof,avoided_CO2,cars,Trees
2355,16793.86,15891.129319,902.730681,90031,34.085784,-118.206666,4792.185,3.195,CA,0.15384,557,6684,15.454273,737.230518,1028.267644,18430.762941,5.38,Los Angeles,"Federal PV Tax Credit, LADWP PV EPBB, Federal...",36,74%,5.8K,5.1M,72.2,110K,388,5.5,8.7K,30.1K,6.4K,772K


In [134]:
df_finances.to_csv('./sun_models_df.csv')

In [132]:
df_finances = df_finances.round(2)

In [1]:
#df_finances